In [1]:
library("coloc")
suppressMessages(suppressWarnings(library(tidyverse)))
library(locuscomparer)
library(susieR)
set.seed(1)
suppressMessages(library('Rfast'))

This is coloc version 5.1.1

Warning message:
"package 'Rfast' was built under R version 4.1.3"
Warning message:
"package 'Rcpp' was built under R version 4.1.3"
Warning message:
"package 'RcppZiggurat' was built under R version 4.1.3"


In [3]:
### Input prep

In [2]:
Metasoft_results <-  read.table("../../ANALYSIS/META/METASOFT/SECOND_PASS/TRANSANCESTRY_METASOFT_SECONDPASS_RESULTS.TAB",fill=TRUE,row.names=NULL,header=FALSE)
META_THREE <- read.table(gzfile("../data/TRANSANCESTRY_GWAS.TAB.gz"),header=T,fill=TRUE,row.names=NULL)
colnames(Metasoft_results) <- c("MARKERNAME","N","PVALUE_FE","BETA_FE","STD_FE","PVALUE_RE","BETA_RE","STD_RE","PVALUE_RE2","STAT1_RE2","STAT2_RE2","PVALUE_BE","I_SQUARE","Q","PVALUE_Q","TAU_SQUARE",
                                "PVALUE_ADIPOGEN","PVALUE_AGEN","PVALUE_METSIM","NA1","NA2","NA3")
Metasoft_results <- Metasoft_results%>%filter(MARKERNAME!="RSID")
TRANSANCESTRY_RESULTS <- merge(META_THREE,Metasoft_results,by="MARKERNAME")
TRANSANCESTRY_RESULTS <- TRANSANCESTRY_RESULTS%>%mutate(TOTAL_N=ceiling(ADIPO_N)+ceiling(AGEN_N)+ceiling(METSIM_N))%>%mutate(MAF_AVG=(ADIPO_EAF*2*ADIPO_N+AGEN_EAF*2*AGEN_N+METSIM_EAF*2*METSIM_N)/(2*TOTAL_N))%>%
filter(ADIPO_EAF>0.005)%>%filter(AGEN_EAF>0.005)%>%filter(METSIM_EAF>0.005)%>%select(-NA1,-N,-NA2,-NA3)
lead <- read.table("../../ANALYSIS/lead_transancestry_3mb_merged.txt",fill=TRUE,header=TRUE)
output_df <- merge(TRANSANCESTRY_RESULTS,lead)%>%select(MARKERNAME,CHROMOSOME,POSITION,METSIM_EA,METSIM_NEA,MAF_AVG,BETA_FE,STD_FE)%>%
rename(rsid=MARKERNAME,chromosome=CHROMOSOME,position=POSITION,allele1=METSIM_EA,allele2=METSIM_NEA,maf=MAF_AVG,beta=BETA_FE,se=STD_FE)%>%mutate(maf = case_when(maf>0.50~ 0.4999,
                            TRUE ~ maf))%>%distinct(rsid,.keep_all = TRUE)

In [3]:
## eQTL files

In [4]:
df_z <- list.files("../../ANALYSIS/COLOC_SUSIE/trans/",pattern='.txt',full.names = TRUE)

In [20]:
for (i in seq_along(df_z))
{
file_name <- paste0(tail(str_split(df_z,"/|.txt")[[i]],n=3)[1],".z")
stats <- read.table(df_z[i],header=T,sep="\t")%>%rename(rsid=SNP)
merged_gwas <- merge(output_df,stats,by="rsid")%>%filter(allele1==A1 && allele2==A2)%>%select(rsid,chromosome,position,allele1,allele2,maf,beta,se) %>% mutate(P=pchisq((beta/se)^2, df=1, lower=F)) %>% 
    distinct %>% group_by(rsid) %>% slice_min(P, n = 1)%>%select(-P)
merged_eqtl <- merge(output_df,stats,by="rsid")%>%filter(allele1==A1 && allele2==A2)%>%select(rsid,chromosome,position,allele1,allele2,maf,b,SE)%>%rename(beta=b,se=SE)%>%
    mutate(P=pchisq((beta/se)^2, df=1, lower=F)) %>% distinct %>% group_by(rsid) %>% slice_min(P, n = 1)%>%select(-P)
write.table(merged_gwas,paste0("../../ANALYSIS/COLOC_SUSIE/harmonized_trans/gwas/",file_name),sep = " ",row.names=FALSE,quote=FALSE)
write.table(merged_eqtl,paste0("../../ANALYSIS/COLOC_SUSIE/harmonized_trans/eqtl/",file_name),sep = " ",row.names=FALSE,quote=FALSE)
}

In [21]:
## EUR

In [29]:
Metasoft_results <-  read.table("../../ANALYSIS/META/METASOFT/SECOND_PASS/EUROPEAN_METASOFT_SECONDPASS_RESULTS.TAB",fill=TRUE,row.names=NULL,header=FALSE)
META_TWO <- read.table(gzfile("../data/EUR_GWAS.TAB.gz"),header=T,fill=TRUE,row.names=NULL)
colnames(Metasoft_results) <- c("MARKERNAME","N","PVALUE_FE","BETA_FE","STD_FE","PVALUE_RE","BETA_RE","STD_RE","PVALUE_RE2","STAT1_RE2","STAT2_RE2","PVALUE_BE","I_SQUARE","Q","PVALUE_Q","TAU_SQUARE",
                                "PVALUE_ADIPOGEN","PVALUE_METSIM","NA1","NA2")
Metasoft_results <- Metasoft_results%>%filter(MARKERNAME!="RSID")
EUR_RESULTS <- merge(META_TWO,Metasoft_results,by="MARKERNAME")
EUR_RESULTS <- EUR_RESULTS%>%mutate(TOTAL_N=ceiling(ADIPO_N)+ceiling(METSIM_N))%>%mutate(MAF_AVG=(ADIPO_EAF*2*ADIPO_N+METSIM_EAF*2*METSIM_N)/(2*TOTAL_N))%>%
filter(ADIPO_EAF>0.005)%>%filter(METSIM_EAF>0.005)%>%select(-NA1,-N,-NA2)
lead <- read.table("../../ANALYSIS/lead_european_3mb_merged.txt",fill=TRUE,header=TRUE)
 output_df <- merge(EUR_RESULTS,lead)%>%select(MARKERNAME,CHROMOSOME.x,POSITION.x,METSIM_EA,METSIM_NEA,MAF_AVG,BETA_FE,STD_FE)%>%
rename(rsid=MARKERNAME,chromosome=CHROMOSOME.x,position=POSITION.x,allele1=METSIM_EA,allele2=METSIM_NEA,maf=MAF_AVG,beta=BETA_FE,se=STD_FE)%>%mutate(maf = case_when(maf>0.50~ 0.4999,
                            TRUE ~ maf))%>%distinct(rsid,.keep_all = TRUE)

In [30]:
## eQTL files

In [31]:
df_z <- list.files("../../ANALYSIS/COLOC_SUSIE/eur/",pattern='.txt',full.names = TRUE)

In [32]:
for (i in seq_along(df_z))
{
file_name <- paste0(tail(str_split(df_z,"/|.txt")[[i]],n=3)[1],".z")
stats <- read.table(df_z[i],header=T,sep="\t")%>%rename(rsid=SNP)
merged_gwas <- merge(output_df,stats,by="rsid")%>%filter(allele1==A1 && allele2==A2)%>%select(rsid,chromosome,position,allele1,allele2,maf,beta,se) %>% mutate(P=pchisq((beta/se)^2, df=1, lower=F)) %>% 
    distinct %>% group_by(rsid) %>% slice_min(P, n = 1)%>%select(-P)
merged_eqtl <- merge(output_df,stats,by="rsid")%>%filter(allele1==A1 && allele2==A2)%>%select(rsid,chromosome,position,allele1,allele2,maf,b,SE)%>%rename(beta=b,se=SE)%>%
    mutate(P=pchisq((beta/se)^2, df=1, lower=F)) %>% distinct %>% group_by(rsid) %>% slice_min(P, n = 1)%>%select(-P)
write.table(merged_gwas,paste0("../../ANALYSIS/COLOC_SUSIE/harmonized_eur/gwas/",file_name),sep = " ",row.names=FALSE,quote=FALSE)
write.table(merged_eqtl,paste0("../../ANALYSIS/COLOC_SUSIE/harmonized_eur/eqtl/",file_name),sep = " ",row.names=FALSE,quote=FALSE)
}

In [ ]:
###METSIM

In [1]:
METSIM_RESULTS <- read.table("../../Summary_Statistic//Formatted//METSIM_hg19_METAL_POS.TAB",header=F,fill=TRUE,row.names=NULL)
colnames(METSIM_RESULTS) <- c("CHROMOSOME","POSITION","MARKERNAME","METSIM_NEA","METSIM_EA","METSIM_N","PVALUE_FE","METSIM_BETA","METSIM_SE")

In [13]:
lead <- read.table("../../ANALYSIS/lead_metsim_3mb_merged.txt",fill=TRUE,header=TRUE)
output_df <- merge(METSIM_RESULTS,lead)%>%mutate(MAF=0.49)%>%select(MARKERNAME,CHROMOSOME,POSITION,METSIM_EA,METSIM_NEA,METSIM_BETA,METSIM_SE,MAF)%>%
rename(rsid=MARKERNAME,chromosome=CHROMOSOME,position=POSITION,allele1=METSIM_EA,allele2=METSIM_NEA,maf=MAF,beta=METSIM_BETA,se=METSIM_SE)%>%distinct(rsid,.keep_all = TRUE)

In [14]:
df_z <- list.files("../../ANALYSIS/COLOC_SUSIE/metsim/",pattern='.txt',full.names = TRUE)

In [19]:
library(stringr)

In [27]:
for (i in seq_along(df_z))
{
file_name <- paste0(tail(str_split(df_z,"/|.txt")[[i]],n=3)[1],".z")
stats <- read.table(df_z[i],header=T,sep="\t")%>%rename(rsid=SNP)
merged_gwas <- merge(output_df,stats,by="rsid")%>%mutate(allele1=gsub("\\_.*", "", allele1))%>%mutate(allele2=gsub("\\_.*", "", allele2))%>%filter(allele1==A1 && allele2==A2)%>%select(rsid,chromosome,position,allele1,allele2,maf,beta,se) %>% mutate(P=pchisq((beta/se)^2, df=1, lower=F)) %>% 
    distinct %>% group_by(rsid) %>% slice_min(P, n = 1)%>%select(-P)
merged_eqtl <- merge(output_df,stats,by="rsid")%>%mutate(allele1=gsub("\\_.*", "", allele1))%>%mutate(allele2=gsub("\\_.*", "", allele2))%>%filter(allele1==A1 && allele2==A2)%>%select(rsid,chromosome,position,allele1,allele2,maf,b,SE)%>%rename(beta=b,se=SE)%>%
    mutate(P=pchisq((beta/se)^2, df=1, lower=F)) %>% distinct %>% group_by(rsid) %>% slice_min(P, n = 1)%>%select(-P)
write.table(merged_gwas,paste0("../../ANALYSIS/COLOC_SUSIE/harmonized_metsim/gwas/",file_name),sep = " ",row.names=FALSE,quote=FALSE)
write.table(merged_eqtl,paste0("../../ANALYSIS/COLOC_SUSIE/harmonized_metsim/eqtl/",file_name),sep = " ",row.names=FALSE,quote=FALSE)
    }